# **Adversarial Robustness of VGG-16, ResNet18, and EfficientNet-B0 on CIFAR-10**
# **Pretrain=True, 30 epoch**

# 

## **1. Import Required Libraries**

In [1]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.get_device_name(0))  # Should print the name of your GPU

True
NVIDIA GeForce RTX 4060


In [4]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchattacks
import time
import pandas as pd
from torchvision import models

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cuda


## **2. Load CIFAR-10 Dataset**

In [5]:

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for model compatibility
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


# **3. Defining Models From torchvision**

## **3.1 Define VGG-16 Model**

In [6]:

# Load pre-defined VGG-16 model from torchvision
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.model = models.vgg16(pretrained=True)
        num_ftrs = self.model.classifier[6].in_features
        self.model.classifier[6] = nn.Linear(num_ftrs, 10)  # Adjust for CIFAR-10 classes

    def forward(self, x):
        return self.model(x)


## **3.2 Define ResNet18 Model**

In [7]:

# Load pre-defined ResNet18 model from torchvision
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.model = models.resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 10)  # Adjust for CIFAR-10 classes

    def forward(self, x):
        return self.model(x)


## **3.3 Define EfficientNet-B0 Model**

In [8]:

# Load pre-defined EfficientNet-B0 model from torchvision
class EfficientNetB0(nn.Module):
    def __init__(self):
        super(EfficientNetB0, self).__init__()
        self.model = models.efficientnet_b0(pretrained=True)
        num_ftrs = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(num_ftrs, 10)  # Adjust for CIFAR-10 classes

    def forward(self, x):
        return self.model(x)



In [9]:
# Initialize models
model_vgg = VGG16().to(device)
model_resnet = ResNet18().to(device)
model_efficient = EfficientNetB0().to(device)

# Define loss function and optimizers
criterion = nn.CrossEntropyLoss()
optimizer_vgg = optim.Adam(model_vgg.parameters(), lr=0.001)
optimizer_resnet = optim.Adam(model_resnet.parameters(), lr=0.001)
optimizer_efficient = optim.Adam(model_efficient.parameters(), lr=0.001)

C:\Users\laika\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\laika\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\laika\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGEN

## **4. Train the Models**

In [10]:

# Training function
def train_model(model, optimizer, model_name, epochs=30):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"{model_name} - Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

print("Training VGG-16 for 30 epochs...")
train_model(model_vgg, optimizer_vgg, "VGG-16")

print("Training ResNet18 for 30 epochs...")
train_model(model_resnet, optimizer_resnet, "ResNet18")

print("Training EfficientNet-B0 for 30 epochs...")
train_model(model_efficient, optimizer_efficient, "EfficientNet-B0")


Training VGG-16 for 30 epochs...
VGG-16 - Epoch 1, Loss: 1.7169
VGG-16 - Epoch 2, Loss: 1.3518
VGG-16 - Epoch 3, Loss: 1.1023
VGG-16 - Epoch 4, Loss: 0.9535
VGG-16 - Epoch 5, Loss: 0.8500
VGG-16 - Epoch 6, Loss: 0.7667
VGG-16 - Epoch 7, Loss: 0.6926
VGG-16 - Epoch 8, Loss: 0.6418
VGG-16 - Epoch 9, Loss: 0.5796
VGG-16 - Epoch 10, Loss: 0.5387
VGG-16 - Epoch 11, Loss: 0.4894
VGG-16 - Epoch 12, Loss: 0.4336
VGG-16 - Epoch 13, Loss: 0.4056
VGG-16 - Epoch 14, Loss: 0.3749
VGG-16 - Epoch 15, Loss: 0.3465
VGG-16 - Epoch 16, Loss: 0.3267
VGG-16 - Epoch 17, Loss: 0.2919
VGG-16 - Epoch 18, Loss: 0.2814
VGG-16 - Epoch 19, Loss: 0.2872
VGG-16 - Epoch 20, Loss: 0.2672
VGG-16 - Epoch 21, Loss: 0.2453
VGG-16 - Epoch 22, Loss: 0.2353
VGG-16 - Epoch 23, Loss: 0.2274
VGG-16 - Epoch 24, Loss: 0.2197
VGG-16 - Epoch 25, Loss: 0.2173
VGG-16 - Epoch 26, Loss: 0.2147
VGG-16 - Epoch 27, Loss: 0.2106
VGG-16 - Epoch 28, Loss: 0.2027
VGG-16 - Epoch 29, Loss: 0.1930
VGG-16 - Epoch 30, Loss: 0.2027
Training ResNet1

In [11]:
torch.save(model_vgg, "model_vgg_30.pth")
torch.save(model_resnet, "model_resnet_30.pth")
torch.save(model_efficient, "model_efficient_30.pth")

## **5. Evaluate Model Accuracy on Clean Test Images**

In [12]:

# Evaluation function
def test_model(model, dataloader, model_name):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'{model_name} Accuracy on clean test images: {accuracy:.2f}%')
    return accuracy

clean_accuracy_vgg = test_model(model_vgg, testloader, "VGG-16")
clean_accuracy_resnet = test_model(model_resnet, testloader, "ResNet18")
clean_accuracy_efficient = test_model(model_efficient, testloader, "EfficientNet-B0")


VGG-16 Accuracy on clean test images: 76.01%
ResNet18 Accuracy on clean test images: 91.30%
EfficientNet-B0 Accuracy on clean test images: 94.04%


# **6. Setting up for evaluating Adversarial attacks**

In [14]:
# Function to evaluate adversarial attacks
def test_adversarial_attack(model, attack, attack_name):
    model.eval()
    correct = 0
    total = 0
    success_count = 0
    start_time = time.time()

    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        adv_images = attack(images, labels)
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        success_count += (predicted != labels).sum().item()

    end_time = time.time()
    attack_time = end_time - start_time
    attack_accuracy = 100 * correct / total
    attack_success_rate = 100 * success_count / total

    print(f'{attack_name} Attack Results:')
    print(f' - Accuracy after attack: {attack_accuracy:.2f}%')
    print(f' - Attack Success Rate: {attack_success_rate:.2f}%')
    print(f' - Time Taken: {attack_time:.2f} seconds\n')

    return attack_accuracy, attack_success_rate, attack_time

# **6.1 Evaluating Adversarial Robustness of VGG-16**

In [13]:
# Define adversarial attacks
attacks = {
    "FGSM": torchattacks.FGSM(model_vgg, eps=0.03),
    "PGD": torchattacks.PGD(model_vgg, eps=0.03, alpha=0.01, steps=40),
    "DeepFool": torchattacks.DeepFool(model_vgg, steps=50),
    "BIM": torchattacks.BIM(model_vgg, eps=0.03, alpha=0.01, steps=10),
    "C&W": torchattacks.CW(model_vgg, c=1, kappa=0, steps=100, lr=0.01)
}

In [15]:
# Run attacks and collect results
attack_results = {}

for attack_name, attack in attacks.items():
    attack_results[attack_name] = test_adversarial_attack(model_vgg, attack, attack_name)

# Print summary
print("\nFinal Adversarial Attack Summary:")
for attack, (acc, success_rate, time_taken) in attack_results.items():
    print(f"{attack}:")
    print(f" - Accuracy After Attack: {acc:.2f}%")
    print(f" - Attack Success Rate: {success_rate:.2f}%")
    print(f" - Time Taken: {time_taken:.2f} sec\n")

FGSM Attack Results:
 - Accuracy after attack: 30.84%
 - Attack Success Rate: 69.16%
 - Time Taken: 1136.87 seconds

PGD Attack Results:
 - Accuracy after attack: 0.08%
 - Attack Success Rate: 99.92%
 - Time Taken: 55678.39 seconds

DeepFool Attack Results:
 - Accuracy after attack: 1.24%
 - Attack Success Rate: 98.76%
 - Time Taken: 50343.89 seconds

BIM Attack Results:
 - Accuracy after attack: 2.86%
 - Attack Success Rate: 97.14%
 - Time Taken: 14308.26 seconds

C&W Attack Results:
 - Accuracy after attack: 0.76%
 - Attack Success Rate: 99.24%
 - Time Taken: 102869.43 seconds


Final Adversarial Attack Summary:
FGSM:
 - Accuracy After Attack: 30.84%
 - Attack Success Rate: 69.16%
 - Time Taken: 1136.87 sec

PGD:
 - Accuracy After Attack: 0.08%
 - Attack Success Rate: 99.92%
 - Time Taken: 55678.39 sec

DeepFool:
 - Accuracy After Attack: 1.24%
 - Attack Success Rate: 98.76%
 - Time Taken: 50343.89 sec

BIM:
 - Accuracy After Attack: 2.86%
 - Attack Success Rate: 97.14%
 - Time Taken

# **6.2 Evaluating Adversarial Robustness of ResNet18**

In [16]:
# Define adversarial attacks
attacks = {
    "FGSM": torchattacks.FGSM(model_resnet, eps=0.03),
    "PGD": torchattacks.PGD(model_resnet, eps=0.03, alpha=0.01, steps=40),
    "DeepFool": torchattacks.DeepFool(model_resnet, steps=50),
    "BIM": torchattacks.BIM(model_resnet, eps=0.03, alpha=0.01, steps=10),
    "C&W": torchattacks.CW(model_resnet, c=1, kappa=0, steps=100, lr=0.01)
}

In [17]:
# ReRun Resnet attacks and collect results
attack_results = {}

for attack_name, attack in attacks.items():
    attack_results[attack_name] = test_adversarial_attack(model_resnet, attack, attack_name)

# Print summary
print("\nFinal Adversarial Attack Summary on ResNet18:")
for attack, (acc, success_rate, time_taken) in attack_results.items():
    print(f"{attack}:")
    print(f" - Accuracy After Attack: {acc:.2f}%")
    print(f" - Attack Success Rate: {success_rate:.2f}%")
    print(f" - Time Taken: {time_taken:.2f} sec\n")

FGSM Attack Results:
 - Accuracy after attack: 22.13%
 - Attack Success Rate: 77.87%
 - Time Taken: 490.19 seconds

PGD Attack Results:
 - Accuracy after attack: 0.08%
 - Attack Success Rate: 99.92%
 - Time Taken: 15480.17 seconds

DeepFool Attack Results:
 - Accuracy after attack: 16.75%
 - Attack Success Rate: 83.25%
 - Time Taken: 32703.17 seconds

BIM Attack Results:
 - Accuracy after attack: 7.18%
 - Attack Success Rate: 92.82%
 - Time Taken: 2272.65 seconds

C&W Attack Results:
 - Accuracy after attack: 3.40%
 - Attack Success Rate: 96.60%
 - Time Taken: 34949.77 seconds


Final Adversarial Attack Summary on ResNet18:
FGSM:
 - Accuracy After Attack: 22.13%
 - Attack Success Rate: 77.87%
 - Time Taken: 490.19 sec

PGD:
 - Accuracy After Attack: 0.08%
 - Attack Success Rate: 99.92%
 - Time Taken: 15480.17 sec

DeepFool:
 - Accuracy After Attack: 16.75%
 - Attack Success Rate: 83.25%
 - Time Taken: 32703.17 sec

BIM:
 - Accuracy After Attack: 7.18%
 - Attack Success Rate: 92.82%
 - 

# **6.3 Evaluating Adversarial Robustness of EfficientNet-B0**

In [18]:
# Define adversarial attacks
attacks = {
    "FGSM": torchattacks.FGSM(model_efficient, eps=0.03),
    "PGD": torchattacks.PGD(model_efficient, eps=0.03, alpha=0.01, steps=40),
    "DeepFool": torchattacks.DeepFool(model_efficient, steps=50),
    "BIM": torchattacks.BIM(model_efficient, eps=0.03, alpha=0.01, steps=10),
    "C&W": torchattacks.CW(model_efficient, c=1, kappa=0, steps=100, lr=0.01)
}

In [19]:
# ReRun attacks and collect results
attack_results = {}

for attack_name, attack in attacks.items():
    attack_results[attack_name] = test_adversarial_attack(model_efficient, attack, attack_name)

# Print summary
print("\nFinal Adversarial Attack Summary on EfficientNet-B0:")
for attack, (acc, success_rate, time_taken) in attack_results.items():
    print(f"{attack}:")
    print(f" - Accuracy After Attack: {acc:.2f}%")
    print(f" - Attack Success Rate: {success_rate:.2f}%")
    print(f" - Time Taken: {time_taken:.2f} sec\n")

FGSM Attack Results:
 - Accuracy after attack: 19.88%
 - Attack Success Rate: 80.12%
 - Time Taken: 303.81 seconds

PGD Attack Results:
 - Accuracy after attack: 0.00%
 - Attack Success Rate: 100.00%
 - Time Taken: 9531.37 seconds

DeepFool Attack Results:
 - Accuracy after attack: 21.89%
 - Attack Success Rate: 78.11%
 - Time Taken: 17870.39 seconds

BIM Attack Results:
 - Accuracy after attack: 0.77%
 - Attack Success Rate: 99.23%
 - Time Taken: 2338.60 seconds

C&W Attack Results:
 - Accuracy after attack: 0.00%
 - Attack Success Rate: 100.00%
 - Time Taken: 14945.31 seconds


Final Adversarial Attack Summary on EfficientNet-B0:
FGSM:
 - Accuracy After Attack: 19.88%
 - Attack Success Rate: 80.12%
 - Time Taken: 303.81 sec

PGD:
 - Accuracy After Attack: 0.00%
 - Attack Success Rate: 100.00%
 - Time Taken: 9531.37 sec

DeepFool:
 - Accuracy After Attack: 21.89%
 - Attack Success Rate: 78.11%
 - Time Taken: 17870.39 sec

BIM:
 - Accuracy After Attack: 0.77%
 - Attack Success Rate: 99